## Q1. Getting the embeddings model

In [1]:
from sentence_transformers import SentenceTransformer

In [2]:
embedding_model = SentenceTransformer('multi-qa-distilbert-cos-v1')

In [3]:
user_question = "I just discovered the course. Can I still join it?"

In [55]:
vector = embedding_model.encode(user_question)

In [35]:
len(vector)

768

In [5]:
vector[0]

0.07822262

What's the first value of the resulting vector?
- **0.07**

In [6]:
import requests 

In [7]:
base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = [doc for doc in docs_response.json() if doc['course'] == 'machine-learning-zoomcamp']

In [8]:
len(documents)

375

## Q2. Creating the embeddings

In [9]:
import numpy as np
from tqdm.auto import tqdm

In [10]:
embeddings = [embedding_model.encode('{question} {text}'.format(**doc)) for doc in tqdm(documents)]

  0%|          | 0/375 [00:00<?, ?it/s]

In [11]:
X = np.array(embeddings)

In [15]:
np.save('X.npy', X) # Save the array for later use

In [12]:
X.shape

(375, 768)

## Q3. Creating the embeddings

In [13]:
scores = X.dot(vector)

In [14]:
scores.max()

0.6506574

What's the highest score in the results?
- **0.65**

### Vector search

In [16]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=documents, embeddings=X)
search_engine.search(vector, num_results=5)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you c

## Q4. Hit-rate for our search engine

In [17]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [20]:
ground_truth[0]

{'question': 'Where can I sign up for the course?',
 'course': 'machine-learning-zoomcamp',
 'document': '0227b872'}

In [51]:
def relevance_total(ground_truth, search_function):
    for q in tqdm(ground_truth):
        doc_id = q['document']
        question_vector = embedding_model.encode(q['question'])
        results = search_function(question_vector)
        relevance = [d['id'] == doc_id for d in results]
        yield relevance

In [73]:
relevance_total_ours = list(relevance_total(ground_truth,
                                            lambda q : search_engine.search(q, num_results=5)))

  0%|          | 0/1830 [00:00<?, ?it/s]

In [74]:
def hit_rate(relevance_total):
    return sum(True in line for line in relevance_total) / len(relevance_total)

In [75]:
hit_rate(relevance_total_ours)

0.9398907103825137

What did you get?
- **0.93**

## Q5. Indexing with Elasticsearch

In [33]:
from elasticsearch import Elasticsearch

In [34]:
es_client = Elasticsearch('http://localhost:9200') 
es_client.info()

ObjectApiResponse({'name': 'b69b8164b0a2', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'EeWAVBapSFi4kebk__TEGQ', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [36]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} ,
            "qa_vector": {"type": "dense_vector",
                            "dims": 768,
                            "index": True,
                            "similarity": "cosine"},
        }
    }
}

index_name = "course-questions_homework"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions_homework'})

In [37]:
documents[0]

{'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
 'section': 'General course-related questions',
 'question': 'How do I sign up?',
 'course': 'machine-learning-zoomcamp',
 'id': '0227b872'}

In [41]:
for doc, vec in tqdm(zip(documents, embeddings)):
    doc['qa_vector'] = vec

In [44]:
for doc in tqdm(documents):
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

  0%|          | 0/375 [00:00<?, ?it/s]

In [56]:
def elastic_search_knn(field, vector, course):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "filter": {
            "term": {
                "course": course
            }
        }
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [68]:
elastic_search_knn('qa_vector', vector, 'machine-learning-zoomcamp')

[{'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'section': 'General course-related questions',
  'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'id': 'ee58a693'},
 {'question': 'I just joined. What should I do next? How can I access course materials?',
  'course': 'machine-learning-zoomcamp',
  'section': 'General course-related questions',
  'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the vid

What's the ID of the document with the highest score?
- **ee58a693**

## Q6. Hit-rate for Elasticsearch

In [69]:
def es_search_function(vector):
    return elastic_search_knn('qa_vector', vector, 'machine-learning-zoomcamp')

relevance_total_es = list(relevance_total(ground_truth,es_search_function))

  0%|          | 0/1830 [00:00<?, ?it/s]

In [76]:
hit_rate(relevance_total_es)

0.9398907103825137

What's hitrate for our dataset for Elastic?
- **0.93**